# 코드 한번에 실행하면 오류가 발생하므로 하나씩 실행시키기

In [1]:
# 네이버 데이터랩 홈페이지 접속하기 
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
import sys
import os
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

driver = webdriver.Chrome('C:/Myexam/chromedriver/chromedriver.exe') 
url = 'https://datalab.naver.com/local/card_result.naver'
driver.get(url)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

C:\Users\user\AppData\Local\Temp/ipykernel_8776/1719304812.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Myexam/chromedriver/chromedriver.exe')


In [2]:
# 전체 선택 버튼 누르기(처음엔 선택이 안 되어있음)

choose_all = driver.find_element(By.CSS_SELECTOR,'#item_2_all').click()

In [3]:
# 이후 조회하기 버튼 누르면 서울 전 자치구에 대한 카드사용량 그래프가 나타난다

lookup = driver.find_element(By.CSS_SELECTOR,'#content > div.section_trend > div.com_box_inner > form > fieldset > a').click()

In [4]:
contents = driver.find_elements(By.CSS_SELECTOR,'#content > div.section_graph > div.com_box_inner > div > div.graph_cate.full > .cate_list')
len(contents)

10

In [5]:
contents[0].find_element(By.CSS_SELECTOR,'div > label').text # 잘 나오는 것 확인

'강남구'

In [6]:
districts = []
for content in contents:
    district = content.find_element(By.CSS_SELECTOR,'div > label').text
    districts.append(district)
    
print(districts)

['강남구', '마포구', '서초구', '영등포구', '중구', '종로구', '송파구', '용산구', '강서구', '관악구']


In [7]:
# 상위 3개의 구 슬라이스
# 리스트 순서가 카드 사용이 

districts_top3 = districts[:3]

print(districts_top3)

['강남구', '마포구', '서초구']


In [8]:
driver.quit() # 상위 3개 자치구 선정 끝

In [9]:
# 크롬 드라이버 

chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)

C:\Users\user\AppData\Local\Temp/ipykernel_8776/2911649310.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path)


In [51]:
def extract():
    for i in range(1,3):
        for j in range(1,3):
            # 가게 이름
            place_title = 'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child('+str(i)+') > div:nth-child('+str(j)+') > figure > figcaption > div > a > h2'
            title = driver.find_element(By.CSS_SELECTOR, place_title)
            # 수집 확인
            print(title.text)


            # 가게 점수
            place_score = 'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child('+str(i)+') > div:nth-child('+str(j)+') > figure > figcaption > div > strong'
            score = driver.find_element(By.CSS_SELECTOR, place_score)
            # 수집 확인
            print(score.text)


            # 가게 위치 및 카테고리
            place_address_category = 'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child('+str(i)+') > div:nth-child('+str(j)+') > figure > figcaption > div > p.etc'
            address_category = driver.find_element(By.CSS_SELECTOR, place_address_category)
            temp = address_category.text
            temp_list = re.split(' - ', temp)
            address = temp_list[0]
            category = temp_list[1]
            #수집 확인
            print(address)
            print(category)


            # 가게 리뷰 수
            place_evaluation = 'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child('+str(i)+') > div:nth-child('+str(j)+') > figure > figcaption > div > p.etc_info > span.review_count'
            evealuation = driver.find_element(By.CSS_SELECTOR, place_evaluation)
            # 수집 확인
            print(evealuation.text)


            # 가게 조회 수
            place_view = 'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child('+str(i)+') > div:nth-child('+str(j)+') > figure > figcaption > div > p.etc_info > span.view_count'
            view = driver.find_element(By.CSS_SELECTOR, place_view)
            # 수집 확인
            print(view.text)

In [52]:
dataset = districts_top3
print(dataset)

['강남구', '마포구', '서초구']


In [53]:
# 데이터 셋의 1~10 페이지를 순회하며 추출합니다.
# 현재 각 페이지의 첫번째 가게만 추출합니다.

dataset_gangnam = []
dataset_mapo = []
dataset_seocho = []

for i in dataset:
    for j in range(1,11):
        # {0} : 검색어(;지역), {1} : 페이지 입니다.
        driver.get(" https://www.mangoplate.com/search/{0}?keyword={0}&page={1}".format(i, j))
        extract()

미라이
4.8
가로수길
이자카야 / 오뎅 / 꼬치
122
456,913
시라카와
4.8
신사/압구정
이자카야 / 오뎅 / 꼬치
67
279,419
페리지
4.8
삼성동
기타 양식
35
24,921
맛짱조개
4.8
논현동
해산물 요리
59
111,281
껠끄쇼즈
4.6
대치동
카페 / 디저트
140
164,871
가디록
4.6
신사/압구정
이탈리안
59
142,505
농민백암왕순대 (강남역점)
4.6
강남역
탕 / 찌개 / 전골
269
483,283
보타르가
4.6
신사/압구정
이탈리안
51
64,460
이치에
4.5
신사/압구정
이자카야 / 오뎅 / 꼬치
171
612,491
봉밀가
4.5
삼성동
국수 / 면 요리
125
65,746
칙피스
4.5
가로수길
세계음식 기타
186
145,609
롸카두들내쉬빌핫치킨 (압구정점)
4.5
신사/압구정
브런치 / 버거 / 샌드위치
105
110,883
스기타마
4.5
청담동
이자카야 / 오뎅 / 꼬치
18
44,735
사브서울
4.5
신사/압구정
칵테일 / 와인
16
9,272
따띠따띠
4.5
신사/압구정
퓨전 양식
12
14,589
판동면옥
4.5
대치동
국수 / 면 요리
17
9,577
중앙해장
4.4
대치동
탕 / 찌개 / 전골
305
404,119
장스테이크
4.4
신사/압구정
스테이크 / 바베큐
150
377,193
밍글스
4.4
청담동
퓨전 한식
74
110,569
경천애인 2237 (삼성점)
4.4
역삼/선릉
고기 요리
161
496,762
농민백암순대 (본점)
4.4
역삼/선릉
탕 / 찌개 / 전골
190
353,634
가온
4.4
신사/압구정
한정식 / 백반 / 정통 한식
20
67,567
호루몬
4.4
신사/압구정
이자카야 / 오뎅 / 꼬치
40
46,284
쇼토
4.4
가로수길
베이커리
79
48,312
고급오복수산
4.4
신사/압구정
정통 일식 / 일반 일식
52
51,530
비플레이트 바이 브라운브레드
4.4
도곡동
베이커리
74
150,256
하쿠시 (휴업중)
4.4
논현동
정통 일식 / 일반 